<a href="https://colab.research.google.com/github/zahraDehghanian97/ASR_Vosk/blob/master/Persian_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install wget pydub wave tqdm vosk
!apt-get ffmpeg

# Download Vosk model
!mkdir models
!wget -P models/ https://alphacephei.com/vosk/models/vosk-model-fa-0.5.zip
!unzip models/vosk-model-fa-0.5.zip -d models/ && rm models/vosk-model-fa-0.5.zip

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 3.7 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=6886905ac41a7343850509e0a2412143fb9909acb7f4a6ecac134990b8d48fd4
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
  Created wheel for wave: filename=Wave-0.0.2-py3-none-any.whl size=1219 sha256=dc44f1304e2b735d2ddae14a5bc588df671d131dd9ecf8f0884e31ca44a46033
  Stored in directory: /root/.cache/pip/wheels/f8/24/4d/1b01c0e32da3eb3fd71bbbc6093fcc557ec3b2d9e532ecd65d
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22427 sha256=60c04b2d0365dfdc34e1d7a854bc8518041772e172e803ebedce7e8b163eb0dc
  Stored in directory: /root/.cache/pip/wheels/d7/31/a1/18e1e7e8bfdafd1

# 1. Convert mp3 to wave format

In [2]:
from pydub import AudioSegment
import os

def mp3_to_wav(source, skip=0, excerpt=False):

    sound = AudioSegment.from_mp3(source) # load source
    sound = sound.set_channels(1) # mono
    sound = sound.set_frame_rate(16000) # 16000Hz

    if excerpt:
        excrept = sound[skip*1000:skip*1000+30000] # 30 seconds - Does not work anymore when using skip
        output_path = os.path.splitext(source)[0]+"_excerpt.wav"
        excrept.export(output_path, format="wav")
    else:
        audio = sound[skip*1000:]
        output_path = os.path.splitext(source)[0]+".wav"
        audio.export(output_path, format="wav")

    return output_path

In [9]:
wave_file = mp3_to_wav('/content/01.mp3',0,False) # please specify here the path to your mp3 file

# 2. Transcribe audio

In [4]:
from vosk import Model, KaldiRecognizer, SetLogLevel
from tqdm.notebook import tqdm
import wave
import os
import json

def transcript_file(input_file, model_path):

    # Check if file exists
    if not os.path.isfile(input_file):
        raise FileNotFoundError(os.path.basename(input_file) + " not found")

    # Check if model path exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(os.path.basename(model_path) + " not found")

    # open audio file
    wf = wave.open(input_file, "rb")

    # check if wave file has the right properties
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
        raise TypeError("Audio file must be WAV format mono PCM.")

    # Initialize model
    model = Model(model_path)
    rec = KaldiRecognizer(model, wf.getframerate())

    # Get file size (to calculate progress bar)
    file_size = os.path.getsize(input_file)

    # Run transcription
    pbar = tqdm(total=file_size)

    # To store our results
    transcription = []

    while True:
        data = wf.readframes(4000) # use buffer of 4000
        pbar.update(len(data))
        if len(data) == 0:
            pbar.set_description("Transcription finished")
            break
        if rec.AcceptWaveform(data):
            # Convert json output to dict
            result_dict = json.loads(rec.Result())
            # Extract text values and append them to transcription list
            transcription.append(result_dict.get("text", ""))

    # Get final bits of audio and flush the pipeline
    final_result = json.loads(rec.FinalResult())
    transcription.append(final_result.get("text", ""))

    transcription_text = ' '.join(transcription)

    return transcription_text

In [10]:
transcription = transcript_file(wave_file, '/content/models/vosk-model-fa-0.5')

  0%|          | 0/26858092 [00:00<?, ?it/s]

In [13]:
print(transcription)

است که توسط عده وزارت ورزش و جوانان پیشنهاد شده استخدام تعدادی بن مایه های موردی داریم قانونی که تصویب شد و با مدل ملی پارا المپیک کرد که ملی المپیک منتظر شد و در قانون شد او از آسمان قشم یا تصویری از بحث و بررسی که نسخه اش شهرت پنج مرد خفاشی دادگاه رسید حالا دوستان دعوا و یا اصلاحات داری خوب انجام دادن به اعتصاب رهبر خواهش می کنم لطف بعد دم در او و این تغییرات که چقدر دوام منطبق با با به به نبی است و منشورها  بین وقتی هم نیز تشکیل شده اید افسری گرگ بالا اصلاح کنید یعنی به تولید و به دعوت برخورد بد باشد البته در کل یال دوستم زاد کشورم هایی دیده باشید ولی به فراگرد این اصلاحیه ندارید و من که امروز باید همان فرایندی که اراده اصل اساسنامه رعایت شده بوده و با رعایت شغل بحث آیا قراردادها تاجیک متن بسم الله الرحمان الرحیم صدای معمول دهید آیا دو برای صدای شما را در من بفرمایید او مرحمت شما و همکاران علی عماد در این جلسه السلام دارم صبح بخیر و ممنون از تشکیل این جلسه و بیدادگری جناب‌عالی حال همانطور که همه ما اشاره فرمودید این اساسنامه کمیته ملی المپیک بعد از ر تصویب از سوی دولت ابلاغ شد و الا